In [17]:
import pandas as pd
import os

df1 = pd.read_csv("output_files/output.csv")
df2 = pd.read_csv("output_files/primary_pdf_analysis.csv")
del df1["Unnamed: 0"]
del df2["Unnamed: 0"]
merged_df = pd.merge(df1, df2, on="DOI", how="outer")

In [18]:
# Create separate DataFrames for matched and unmatched elements
matched_df = merged_df.dropna(subset=["Col1", "first_location"])  # Replace "column_name" with the actual column name containing the merged values
matched_df = matched_df.loc[:, ~matched_df.columns.str.contains("location")]

unmatched_df1 = merged_df[merged_df["Col1"].isnull()]  # Replace "column_name" with the actual column name containing the merged values
unmatched_df1 = unmatched_df1.drop(columns=["Col1", "Col2", "Col3", "Col4", "Col5", "Col6", "Col7"])

unmatched_df2 = merged_df[(merged_df["first_location"].isnull()) & (merged_df["Paper Title"].isnull())]
unmatched_df2 = unmatched_df2[["DOI", "Col1", "Col2", "Col3", "Col4", "Col5", "Col6", "Col7"]]

In [19]:
print(matched_df.shape)
print(unmatched_df1.shape)
print(unmatched_df2.shape)


(150, 18)
(163, 15)
(21, 8)


In [20]:
# Directory path
directory = "ArcGIS"

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Save DataFrames to CSV files
unmatched_df1.to_csv(os.path.join(directory, "scopus_data.csv"), index=False)
unmatched_df2.to_csv(os.path.join(directory, "pdf_data.csv"), index=False)
matched_df.to_csv(os.path.join(directory, "clear_data.csv"), index=False)


In [54]:
import pandas as pd

# Read the dataframe from your source file
df = matched_df

# Read the country list from "country_list.csv"
country_list = pd.read_csv("country_list/country_list.csv")

# List of country names other than the US
countries_to_check = country_list["Country"].tolist()

# List of forms of the US (add more if needed)
us_forms = ["US", "United States", "USA", "Mexico"]

# Create an empty dataframe to store the discarded rows
discarded_df = pd.DataFrame(columns=df.columns)

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    should_discard = False
    
    # Check if any country other than the US is mentioned
    for country in countries_to_check:
        if any(isinstance(item, str) and country in item for item in row.values[11:]):
            # Check if the US is mentioned in any form
            us_mentioned = any(isinstance(item, str) and any(us_form in item for us_form in us_forms) for item in row.values[11:])

            # If the US is not mentioned, discard the row
            if not us_mentioned:
                should_discard = True
                break
    
    if should_discard:
        # Add the discarded row to the discarded dataframe
        discarded_df = pd.concat([discarded_df, row.to_frame().T])
        
        # Drop the row from the original dataframe
        df = df.drop(index)
        
# # Save the discarded rows in a separate file
# discarded_df.to_csv("discarded.csv", index=False)

# # Optional: Save the modified dataframe without the discarded rows
# df.to_csv("modified_dataframe.csv", index=False)


In [55]:
df

,Paper Title,Scopus ID,DOI,URL,Lead Author,Affiliation,Author count,Area of Study,Publication,Cover Date,Number of citations,Col1,Col2,Col3,Col4,Col5,Col6,Col7
61,Assessing salinization of coastal groundwater ...,SCOPUS_ID:85107405136,10.1016/j.ejrh.2021.100842,https://api.elsevier.com/content/abstract/scop...,Ramatlapeng G.J.,Université de Yaoundé I,5.0,Water Science and Technology & Earth and Plane...,Journal of Hydrology: Regional Studies,2021-08-01,5.0,′′ (21),Youpwe (14),Douala (13),Mirzavand (6),N (6),Shalem (4),Wouri (4)
64,Deltas in arid environments,SCOPUS_ID:85110251391,10.3390/w13121677,https://api.elsevier.com/content/abstract/scop...,Day J.,East China Normal University,6.0,"Geography, Planning and Development & Biochemi...",Water (Switzerland),2021-06-02,3.0,the Colorado River (13),Mexico (12),Colorado (7),the Gulf of California (7),Nile (6),Egypt (6),Indus (5)
165,Flood hazard analysis of proposed regulator on...,SCOPUS_ID:85077149389,10.3390/hydrology6030080,https://api.elsevier.com/content/abstract/scop...,Hamdan A.N.A.,University of Basrah,3.0,Oceanography & Water Science and Technology & ...,Hydrology,2019-09-01,5.0,the Tigris River (8),Tigris River (7),the Arabian Gulf (6),HEC (6),Basrah Province (3),the River-Surveyor (3),Euphrates (3)
166,Salt marsh migration into salinized agricultur...,SCOPUS_ID:85073487678,10.1111/jvs.12774,https://api.elsevier.com/content/abstract/scop...,Gedan K.,Universidad de Oviedo,2.0,Ecology & Plant Science,Journal of Vegetation Science,2019-09-01,20.0,Phragmites (4),Smith (4),Chesapeake\tBay (3),Washington (2),Maryland (2),Marsh (2),Canoco (2)
168,"Influence of flooding, salinization, and soil ...",SCOPUS_ID:85070485196,10.1021/acs.jafc.9b02947,https://api.elsevier.com/content/abstract/scop...,Redman Z.,"University of California, Davis",4.0,Chemistry (all) & Agricultural and Biological ...,Journal of Agricultural and Food Chemistry,2019-07-09,11.0,California (22),MO (3),Scardaci (3),Sacramento (2),Valley (2),Kansas City (2),Tanji (2)
186,Drivers of landscape changes in coastal ecosys...,SCOPUS_ID:85051621543,10.3390/rs10081280,https://api.elsevier.com/content/abstract/scop...,Jorgenson T.M.,ABR Inc. Environmental Research and Services,3.0,Earth and Planetary Sciences (all),Remote Sensing,2018-08-01,30.0,Alaska (4),Arctic (1),Betula (1),Anchorage (1),GA (1),Grid (1),KME (1)
209,Formation of a new delta-like object in the Ka...,SCOPUS_ID:85012912526,10.1134/S009780781701016X,https://api.elsevier.com/content/abstract/scop...,Starodubtsev V.,National University of Life and Environmental ...,1.0,Water Science and Technology,Water Resources,2017-01-01,4.0,pp. (7),Starodubtsev (6),Ili (4),Kapchagai (3),Moscow (3),Bogdanets (3),the Ili R. (2)
218,Succession of flooded soils of coastal area of...,SCOPUS_ID:85018476682,10.1134/S2079096116020037,https://api.elsevier.com/content/abstract/scop...,Asgherova D.B.,Caspian Institute of Biological Resources DSC RAS,4.0,"Global and Planetary Change & Ecology, Evoluti...",Arid Ecosystems,2016-04-01,0.0,Dagestan (6),pp. (5),Moscow (5),Makhachkala (3),Caspian Sea (3),the Caspian Sea (2),Kaspiiskoe (2)
231,Salinity management in the coastal region of t...,SCOPUS_ID:84929120161,10.1016/j.agwat.2014.08.022,https://api.elsevier.com/content/abstract/scop...,Raats P.A.C.,NaN,1.0,Agronomy and Crop Science & Water Science and ...,Agricultural Water Management,2015-07-01,22.0,Netherlands (26),Zeeland (8),Friesland (5),North Sea (3),Holland (3),Zuur (2),Gelderland (2)
234,The rivers of civilization,SCOPUS_ID:84924664043,10.1016/j.quascirev.2015.02.004,https://api.elsevier.com/content/abstract/scop...,Macklin M.G.,Aberystwyth University,2.0,"Global and Planetary Change & Ecology, Evoluti...",Quaternary Science Reviews,2015-04-05,120.0,Indus (12),Macklin (10),Ashworth (9),Nile (6),Lewin (6),Field (3),Latrubesse (3)
